# Web Scrapping uber eats website
Here, we scrap the data from uber eats website to gather information about restraunts in san francisco. We get the following information for  restraunts in SF across different price ranges. We also scrape the information about their addres, how many ratings they have received, what is their overall rating out of 5 and the cuicine they serve. Our goal is to scrape data from yelp and uber eats and answer the following business question - A real estate agent in Sanfrancisco is considering opening a restaurant and wants to determine the best neighborhood, cuicine and price range for the reataurant. The agent wants to gather the data about existing restaurants in SF and find out the ideal combination of neighborhood, cuicine type and prcie range that would likely attract customers and ensure the success of the restraunt venture in SF.

In [2]:
#Importing required libraries
from bs4 import BeautifulSoup
import requests
import json
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

import re
import pandas as pd
import uuid

import mysql.connector
import warnings

Using Selenium to load the uber eats page and apply filte price - $. After we will be saving the page in the directory 'Uber Eats'. Will be using reading this file later and use beautiful soup and regex to get scrape the website and get data.

In [3]:
#Creating an isntance for chrome webdriver
driver = webdriver.Chrome()

#Navigating to Uber eats
driver.get('https://www.ubereats.com/find-near-me/san-francisco-ca/food') 

time.sleep(5)

dollar_1 = driver.find_element(By.CSS_SELECTOR, '[data-baseweb="button"]')
dollar_1.click()

time.sleep(5)

#creating the directory
import os
directory = 'Uber Eats Web scrapping'

#Checking if the directory exists
if not os.path.exists(directory):
    os.makedirs(directory)

#Saving the web page
html_source = driver.page_source
with open(os.path.join(directory,'Uber_Eats_1_dollar_rest.html'), 'w', encoding='utf-8') as file:
    file.write(html_source)

driver.quit()


In [19]:
#Opening the page and parsing into html.
from bs4 import BeautifulSoup
with open(os.path.join(directory,'Uber_Eats_1_dollar_rest.html'), 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

#Using beautiful soup to extract required information

#Starting with storing all the data for each restaurant in a list object
output_list = []
for x in soup.findAll('div', class_ = 'al br am l0 lw'):
    output = x.text
    output_list.append(output)

#print(output_list)

#Now extracting restaurant name by extracting all the values before the $ sign in each element. We will iterate over each object
retaurant_names = []

for item in output_list:
    #find the position of dollar sign
    dollar_index = item.find('$')
    
    if dollar_index != -1:
        #Extract the restuarant name before '$' sign
        retaurant_name = item[:dollar_index].strip()
        retaurant_names.append(retaurant_name)

#print(retaurant_names)
print(len(retaurant_names))


#Now extracting address by extracting information between 'Location marker' and 'Delivery..'. We will iterate over each object
addresses = []

for item in output_list:
    #find the position of Location Marker
    lm_index = item.find('Location marker')
    
    if lm_index != -1:
        #find the position of 'Delivery'
        delivery_index = item.find('Delivery')
        #Extract the address info
        addresse = item[lm_index + len('Location Marker') :delivery_index].strip()
        addresses.append(addresse)

#print(addresses)
print(len(addresses))

#Now extracting zipcodes from addresses using regex
zipcodes = []
zip_pattern = r'\b\d{5}(?:-\d{4})?\b'

for address in addresses:
    match = re.search(zip_pattern, address)
    
    if match:
        zipcodes.append(match.group())
    else:
        zipcodes.append(0)
        
#print(zipcodes)
print(len(zipcodes))

#Now fetching the details of review_count using regex
reviews = []

review_pattern = re.compile(r'\((\d+\+?) ratings\)')

#Iterate over output_list
for element in output_list:
    #Search for 'ratings' in the element
    match = review_pattern.search(element)
    if match:
        #If ratings found, extract the number within the brackets
        review = match.group(1)
        reviews.append(review)
    else:
        #If rating not found, return 'new'
        reviews.append('new')

#print(reviews)
print(len(reviews))

#Now extracting cuisine form this list
cuisine_list = []

#Iterate over each element in output_list
for element in output_list:
    #Search for the substring after 'Circle i' using regex
    match = re.search(r'Circle i(\w+)(?=\W|$)', element)
    if match:
        cuicine_info = match.group(1)
        #split the cuicine info by non-alphanumeric characters and filter out empty strings
        cuisines = [c.strip() for c in re.split(r'\W+', cuicine_info) if c.strip()]
        cuisine_list.append(cuisines)
   
#print(cuisine_list)
print(len(cuisine_list))

#Now extracting the rating informaation using regex 
rating_pattern = r'(\d+(\.\d+)?)\s*\([0-9]+\s*ratings\)'

ratings = []
#Iterate over output_list
for element in output_list:
    match = re.search(rating_pattern, element)
    if match:
        rating = match.group(1)
        ratings.append(rating)
    else:
        ratings.append('0')

#print(ratings)
print(len(ratings))


300
300
300
300
300
300


In [20]:
#Creating a data dictionary

#Creating unique id
id = [uuid.uuid4().hex[:10] for _ in range(300)]

data_dict = {
    'id': id,
    'name' : retaurant_names,
    'Address' : addresses,
    'city': 'San Francisco',
    'zip_code': zipcodes,
    'review_count': reviews,
    'categories': cuisine_list,
    'rating': ratings,
    'price': '$'

}

data = dict(zip(data_dict.keys(), zip(*data_dict.values())))

print(data)

{'id': ('0df63a9290', 'Taqueria Cazadores', '1600 Mission St, San Francisco, CA 94103', 'S', '94103', '32', ['Mexican'], '4.7', '$')}


In [28]:
# Function to create a SQL table
def create_sql_table(db_name, table_name, table_info):
    try:
        # Connect to server
        conn = mysql.connector.connect(host='127.0.0.1',
                                       port = '3306',
                                       user='root',
                                       password='Pvv@399051')
        cursor = conn.cursor()
        
        # Create database if it doesn't exist
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        
        # Create table if it doesn't exist
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {db_name}.{table_name} {table_info}")
        
    except mysql.connector.Error as err:
        print("Something went wrong:", err)
    finally:
        cursor.close()
        conn.close()

# Function to insert data into the table
def insert_data_dict(db_name, table_name, data_dict):
    try:
        #Connect to MySQL database
        conn = mysql.connector.connect(host='127.0.0.1',
                                       port = '3306',
                                       user='root',
                                       password='Pvv@399051', database=db_name)
        cursor = conn.cursor()
        
        #Find the max length among all values
        max_length = max(len(lst) if isinstance(lst,list) else 1 for lst in data_dict.values())
        
        #Fill missing values with None or empty string
        for key, value in data_dict.items():
            if not isinstance(value,list):
                data_dict[key] = [value] * max_length
            elif len(value) < max_length:
                data_dict[key].extend([None]*(max_legth - len(values)))
                
        #Create a parameterised sql query
        placeholders = ', '.join(['%s'] * len(data_dict))
        columns = ', '.join(data_dict.keys())
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        
        #Prepare the data values to be inserted
        values = [tuple(data_dict[key][i] for key in data_dict) for i in range(max_length)]
        
        #Execute the query with the values from the data dictionary
        cursor.executemany(sql, values)

        #Commit the transaction
        conn.commit()

    except mysql.connector.Error as err:
        print("Something went wrong:", err)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()


# My database and table name
SQL_DB = "bax422_final_price1"
SQL_TABLE_NAME = "uber_eats_price1"

# SQL table structure
SQL_TABLE_USER_INFO = """
(
    id VARCHAR(100) PRIMARY KEY,
    name VARCHAR(500),
    address1 VARCHAR(500),
    address2 VARCHAR(100),
    city VARCHAR(100),
    zip_code INT,
    review_count VARCHAR(100),
    categories VARCHAR(500),
    rating INT,
    price VARCHAR(10)
    
)
"""

# Create your database and table
create_sql_table(SQL_DB, SQL_TABLE_NAME, SQL_TABLE_USER_INFO)



#Converting dataframe uber_eats_1 to dictionary to  pass it through insert_into function
#uber_eats_1_dict = uber_eats_1.to_dict(orient = 'records')

#Insert data into the databse
for i in range(300):
    data_dict = {
        'id' : id[i],
        'name' : retaurant_names[i],
        'address1' : addresses[i],
        'address2' : '',
        'city' : ' San Francisco',
        'zip_code' : zipcodes[i],
        'review_count' : reviews[i],
        'categories' : cuisine_list[i],
        'rating': ratings[i],
        'price' :'$'

    }
    insert_data_dict(SQL_DB, SQL_TABLE_NAME, data_dict)


## Uber eats 2 dollar restaurants

In [38]:
#Creating an isntance for chrome webdriver
driver = webdriver.Chrome()

#Navigating to Uber eats
driver.get('https://www.ubereats.com/find-near-me/san-francisco-ca/food') 

time.sleep(5)

dollar_2 = driver.find_elements(By.CSS_SELECTOR, "[data-baseweb='button']")
dollar_2[1].click()

time.sleep(5)

#creating the directory
import os
directory = 'Uber Eats Web scrapping'

#Checking if the directory exists
if not os.path.exists(directory):
    os.makedirs(directory)

#Saving the web page
html_source = driver.page_source
with open(os.path.join(directory,'Uber_Eats_2_dollar_rest.html'), 'w', encoding='utf-8') as file:
    file.write(html_source)

driver.quit()



In [40]:
#Opening the page and parsing into html.
from bs4 import BeautifulSoup
with open(os.path.join(directory,'Uber_Eats_2_dollar_rest.html'), 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

#Using beautiful soup to extract required information

#Starting with storing all the data for each restaurant in a list object
output_list_2 = []
for x in soup.findAll('div', class_ = 'al br am l0 lw'):
    output = x.text
    output_list_2.append(output)

#print(output_list_2)

#Now extracting restaurant name by extracting all the values before the $ sign in each element. We will iterate over each object
retaurant_names_2 = []

for item in output_list_2:
    #find the position of dollar sign
    dollar_index = item.find('$')
    
    if dollar_index != -1:
        #Extract the restuarant name before '$' sign
        retaurant_name = item[:dollar_index].strip()
        retaurant_names_2.append(retaurant_name)

#print(retaurant_names)
print(len(retaurant_names_2))


#Now extracting address by extracting information between 'Location marker' and 'Delivery..'. We will iterate over each object
addresses_2 = []

for item in output_list_2:
    #find the position of Location Marker
    lm_index = item.find('Location marker')
    
    if lm_index != -1:
        #find the position of 'Delivery'
        delivery_index = item.find('Delivery')
        #Extract the address info
        addresse = item[lm_index + len('Location Marker') :delivery_index].strip()
        addresses_2.append(addresse)

#print(addresses)
print(len(addresses_2))

#Now extracting zipcodes from addresses using regex
zipcodes_2 = []
zip_pattern = r'\b\d{5}(?:-\d{4})?\b'

for address in addresses_2:
    match = re.search(zip_pattern, address)
    
    if match:
        zipcodes_2.append(match.group())
    else:
        zipcodes_2.append(0)
        
#print(zipcodes)
print(len(zipcodes_2))

#Now fetching the details of review_count using regex
reviews_2 = []

review_pattern = re.compile(r'\((\d+\+?) ratings\)')

#Iterate over output_list
for element in output_list_2:
    #Search for 'ratings' in the element
    match = review_pattern.search(element)
    if match:
        #If ratings found, extract the number within the brackets
        review = match.group(1)
        reviews_2.append(review)
    else:
        #If rating not found, return 'new'
        reviews_2.append('new')

#print(reviews)
print(len(reviews_2))

#Now extracting cuisine form this list
cuisine_list_2 = []

#Iterate over each element in output_list
for element in output_list_2:
    #Search for the substring after 'Circle i' using regex
    match = re.search(r'Circle i(\w+)(?=\W|$)', element)
    if match:
        cuicine_info = match.group(1)
        #split the cuicine info by non-alphanumeric characters and filter out empty strings
        cuisines = [c.strip() for c in re.split(r'\W+', cuicine_info) if c.strip()]
        cuisine_list_2.append(cuisines)
   
#print(cuisine_list)
print(len(cuisine_list_2))

#Now extracting the rating informaation using regex 
rating_pattern = r'(\d+(\.\d+)?)\s*\([0-9]+\s*ratings\)'

ratings_2 = []
#Iterate over output_list
for element in output_list_2:
    match = re.search(rating_pattern, element)
    if match:
        rating = match.group(1)
        ratings_2.append(rating)
    else:
        ratings_2.append('0')

#print(ratings)
print(len(ratings_2))

300
300
300
300
300
300


In [41]:
#Creating a data dictionary

#Creating unique id
id_2 = [uuid.uuid4().hex[:10] for _ in range(300)]

data_dict_2 = {
    'id': id_2,
    'name' : retaurant_names_2,
    'Address' : addresses_2,
    'city': 'San Francisco',
    'zip_code': zipcodes_2,
    'review_count': reviews_2,
    'categories': cuisine_list_2,
    'rating': ratings_2,
    'price': '$$'

}

data = dict(zip(data_dict.keys(), zip(*data_dict.values())))

print(data)


{'id': ('ad92cdf1fc', 'Gluten Free Pizza Garden - SOMA', '60 Morris St, San Francisco, CA 94107', '', ' San Francisco', '94107', 'new', 'PizzaAmericanItalianGluten', '0', '$')}


In [46]:
# Function to create a SQL table
def create_sql_table(db_name, table_name, table_info):
    try:
        # Connect to server
        conn = mysql.connector.connect(host='127.0.0.1',
                                       port = '3306',
                                       user='root',
                                       password='Pvv@399051')
        cursor = conn.cursor()
        
        # Create database if it doesn't exist
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        
        # Create table if it doesn't exist
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {db_name}.{table_name} {table_info}")
        
    except mysql.connector.Error as err:
        print("Something went wrong:", err)
    finally:
        cursor.close()
        conn.close()

# Function to insert data into the table
def insert_data_dict(db_name, table_name, data_dict):
    try:
        #Connect to MySQL database
        conn = mysql.connector.connect(host='127.0.0.1',
                                       port = '3306',
                                       user='root',
                                       password='Pvv@399051', database=db_name)
        cursor = conn.cursor()
        
        #Find the max length among all values
        max_length = max(len(lst) if isinstance(lst,list) else 1 for lst in data_dict.values())
        
        #Fill missing values with None or empty string
        for key, value in data_dict.items():
            if not isinstance(value,list):
                data_dict[key] = [value] * max_length
            elif len(value) < max_length:
                data_dict[key].extend([None]*(max_legth - len(values)))
                
        #Create a parameterised sql query
        placeholders = ', '.join(['%s'] * len(data_dict))
        columns = ', '.join(data_dict.keys())
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        
        #Prepare the data values to be inserted
        values = [tuple(data_dict[key][i] for key in data_dict) for i in range(max_length)]
        
        #Execute the query with the values from the data dictionary
        cursor.executemany(sql, values)

        #Commit the transaction
        conn.commit()

    except mysql.connector.Error as err:
        print("Something went wrong:", err)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()


# My database and table name
SQL_DB = "bax422_final_price1"
SQL_TABLE_NAME = "uber_eats_price2"

# SQL table structure
SQL_TABLE_USER_INFO = """
(
    id VARCHAR(100) PRIMARY KEY,
    name VARCHAR(500),
    address1 VARCHAR(500),
    address2 VARCHAR(100),
    city VARCHAR(100),
    zip_code INT,
    review_count VARCHAR(100),
    categories VARCHAR(500),
    rating INT,
    price VARCHAR(10)
    
)
"""

# Create your database and table
create_sql_table(SQL_DB, SQL_TABLE_NAME, SQL_TABLE_USER_INFO)



#Converting dataframe uber_eats_1 to dictionary to  pass it through insert_into function
#uber_eats_1_dict = uber_eats_1.to_dict(orient = 'records')

#Insert data into the databse
for i in range(300):
    data_dict = {
        'id' : id_2[i],
        'name' : retaurant_names_2[i],
        'address1' : addresses_2[i],
        'address2' : '',
        'city' : ' San Francisco',
        'zip_code' : zipcodes_2[i],
        'review_count' : reviews_2[i],
        'categories' : cuisine_list_2[i],
        'rating': ratings_2[i],
        'price' :'$$'

    }
    insert_data_dict(SQL_DB, SQL_TABLE_NAME, data_dict)


Something went wrong: 1265 (01000): Data truncated for column 'zip_code' at row 1


## Uber eats 3 dollar restaurants

In [48]:
#Creating an isntance for chrome webdriver
driver = webdriver.Chrome()

#Navigating to Uber eats
driver.get('https://www.ubereats.com/find-near-me/san-francisco-ca/food') 

time.sleep(5)

dollar_2 = driver.find_elements(By.CSS_SELECTOR, "[data-baseweb='button']")
dollar_2[2].click()

time.sleep(5)

#creating the directory
import os
directory = 'Uber Eats Web scrapping'

#Checking if the directory exists
if not os.path.exists(directory):
    os.makedirs(directory)

#Saving the web page
html_source = driver.page_source
with open(os.path.join(directory,'Uber_Eats_3_dollar_rest.html'), 'w', encoding='utf-8') as file:
    file.write(html_source)


driver.quit()


In [49]:
#Opening the page and parsing into html.
from bs4 import BeautifulSoup
with open(os.path.join(directory,'Uber_Eats_3_dollar_rest.html'), 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

#Using beautiful soup to extract required information

#Starting with storing all the data for each restaurant in a list object
output_list_3 = []
for x in soup.findAll('div', class_ = 'al br am l0 lw'):
    output = x.text
    output_list_3.append(output)

#print(output_list_3)

#Now extracting restaurant name by extracting all the values before the $ sign in each element. We will iterate over each object
retaurant_names_3 = []

for item in output_list_3:
    #find the position of dollar sign
    dollar_index = item.find('$')
    
    if dollar_index != -1:
        #Extract the restuarant name before '$' sign
        retaurant_name = item[:dollar_index].strip()
        retaurant_names_3.append(retaurant_name)

#print(retaurant_names)
print(len(retaurant_names_3))


#Now extracting address by extracting information between 'Location marker' and 'Delivery..'. We will iterate over each object
addresses_3 = []

for item in output_list_3:
    #find the position of Location Marker
    lm_index = item.find('Location marker')
    
    if lm_index != -1:
        #find the position of 'Delivery'
        delivery_index = item.find('Delivery')
        #Extract the address info
        addresse = item[lm_index + len('Location Marker') :delivery_index].strip()
        addresses_3.append(addresse)

#print(addresses)
print(len(addresses_3))

#Now extracting zipcodes from addresses using regex
zipcodes_3 = []
zip_pattern = r'\b\d{5}(?:-\d{4})?\b'

for address in addresses_3:
    match = re.search(zip_pattern, address)
    
    if match:
        zipcodes_3.append(match.group())
    else:
        zipcodes_3.append(0)
        
#print(zipcodes)
print(len(zipcodes_3))

#Now fetching the details of review_count using regex
reviews_3 = []

review_pattern = re.compile(r'\((\d+\+?) ratings\)')

#Iterate over output_list
for element in output_list_3:
    #Search for 'ratings' in the element
    match = review_pattern.search(element)
    if match:
        #If ratings found, extract the number within the brackets
        review = match.group(1)
        reviews_3.append(review)
    else:
        #If rating not found, return 'new'
        reviews_3.append('new')

#print(reviews)
print(len(reviews_3))

#Now extracting cuisine form this list
cuisine_list_3 = []

#Iterate over each element in output_list
for element in output_list_3:
    #Search for the substring after 'Circle i' using regex
    match = re.search(r'Circle i(\w+)(?=\W|$)', element)
    if match:
        cuicine_info = match.group(1)
        #split the cuicine info by non-alphanumeric characters and filter out empty strings
        cuisines = [c.strip() for c in re.split(r'\W+', cuicine_info) if c.strip()]
        cuisine_list_3.append(cuisines)
   
#print(cuisine_list)
print(len(cuisine_list_3))

#Now extracting the rating informaation using regex 
rating_pattern = r'(\d+(\.\d+)?)\s*\([0-9]+\s*ratings\)'

ratings_3 = []
#Iterate over output_list
for element in output_list_3:
    match = re.search(rating_pattern, element)
    if match:
        rating = match.group(1)
        ratings_3.append(rating)
    else:
        ratings_3.append('0')

#print(ratings)
print(len(ratings_3))

39
39
39
39
39
39


In [50]:
#Creating unique id
id_3 = [uuid.uuid4().hex[:10] for _ in range(39)]

In [51]:
# Function to create a SQL table
def create_sql_table(db_name, table_name, table_info):
    try:
        # Connect to server
        conn = mysql.connector.connect(host='127.0.0.1',
                                       port = '3306',
                                       user='root',
                                       password='Pvv@399051')
        cursor = conn.cursor()
        
        # Create database if it doesn't exist
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        
        # Create table if it doesn't exist
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {db_name}.{table_name} {table_info}")
        
    except mysql.connector.Error as err:
        print("Something went wrong:", err)
    finally:
        cursor.close()
        conn.close()

# Function to insert data into the table
def insert_data_dict(db_name, table_name, data_dict):
    try:
        #Connect to MySQL database
        conn = mysql.connector.connect(host='127.0.0.1',
                                       port = '3306',
                                       user='root',
                                       password='Pvv@399051', database=db_name)
        cursor = conn.cursor()
        
        #Find the max length among all values
        max_length = max(len(lst) if isinstance(lst,list) else 1 for lst in data_dict.values())
        
        #Fill missing values with None or empty string
        for key, value in data_dict.items():
            if not isinstance(value,list):
                data_dict[key] = [value] * max_length
            elif len(value) < max_length:
                data_dict[key].extend([None]*(max_legth - len(values)))
                
        #Create a parameterised sql query
        placeholders = ', '.join(['%s'] * len(data_dict))
        columns = ', '.join(data_dict.keys())
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        
        #Prepare the data values to be inserted
        values = [tuple(data_dict[key][i] for key in data_dict) for i in range(max_length)]
        
        #Execute the query with the values from the data dictionary
        cursor.executemany(sql, values)

        #Commit the transaction
        conn.commit()

    except mysql.connector.Error as err:
        print("Something went wrong:", err)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()


# My database and table name
SQL_DB = "bax422_final_price1"
SQL_TABLE_NAME = "uber_eats_price3"

# SQL table structure
SQL_TABLE_USER_INFO = """
(
    id VARCHAR(100) PRIMARY KEY,
    name VARCHAR(500),
    address1 VARCHAR(500),
    address2 VARCHAR(100),
    city VARCHAR(100),
    zip_code INT,
    review_count VARCHAR(100),
    categories VARCHAR(500),
    rating INT,
    price VARCHAR(10)
    
)
"""

# Create your database and table
create_sql_table(SQL_DB, SQL_TABLE_NAME, SQL_TABLE_USER_INFO)



#Converting dataframe uber_eats_1 to dictionary to  pass it through insert_into function
#uber_eats_1_dict = uber_eats_1.to_dict(orient = 'records')

#Insert data into the databse
for i in range(39):
    data_dict = {
        'id' : id_3[i],
        'name' : retaurant_names_3[i],
        'address1' : addresses_3[i],
        'address2' : '',
        'city' : ' San Francisco',
        'zip_code' : zipcodes_3[i],
        'review_count' : reviews_3[i],
        'categories' : cuisine_list_3[i],
        'rating': ratings_3[i],
        'price' :'$$$'

    }
    insert_data_dict(SQL_DB, SQL_TABLE_NAME, data_dict)


## Uber eats 4 dollar restaurants

In [53]:
#Creating an isntance for chrome webdriver
driver = webdriver.Chrome()

#Navigating to Uber eats
driver.get('https://www.ubereats.com/find-near-me/san-francisco-ca/food') 

time.sleep(5)

dollar_4 = driver.find_elements(By.CSS_SELECTOR, "[data-baseweb='button']")
dollar_4[3].click()

time.sleep(5)

#creating the directory
import os
directory = 'Uber Eats Web scrapping'

#Checking if the directory exists
if not os.path.exists(directory):
    os.makedirs(directory)

#Saving the web page
html_source = driver.page_source
with open(os.path.join(directory,'Uber_Eats_4_dollar_rest.html'), 'w', encoding='utf-8') as file:
    file.write(html_source)


driver.quit()


In [54]:
#Opening the page and parsing into html.
from bs4 import BeautifulSoup
with open(os.path.join(directory,'Uber_Eats_4_dollar_rest.html'), 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

#Using beautiful soup to extract required information

#Starting with storing all the data for each restaurant in a list object
output_list_4 = []
for x in soup.findAll('div', class_ = 'al br am l0 lw'):
    output = x.text
    output_list_4.append(output)

#print(output_list_3)

#Now extracting restaurant name by extracting all the values before the $ sign in each element. We will iterate over each object
retaurant_names_4 = []

for item in output_list_4:
    #find the position of dollar sign
    dollar_index = item.find('$')
    
    if dollar_index != -1:
        #Extract the restuarant name before '$' sign
        retaurant_name = item[:dollar_index].strip()
        retaurant_names_4.append(retaurant_name)

#print(retaurant_names)
print(len(retaurant_names_4))


#Now extracting address by extracting information between 'Location marker' and 'Delivery..'. We will iterate over each object
addresses_4 = []

for item in output_list_4:
    #find the position of Location Marker
    lm_index = item.find('Location marker')
    
    if lm_index != -1:
        #find the position of 'Delivery'
        delivery_index = item.find('Delivery')
        #Extract the address info
        addresse = item[lm_index + len('Location Marker') :delivery_index].strip()
        addresses_4.append(addresse)

#print(addresses)
print(len(addresses_4))

#Now extracting zipcodes from addresses using regex
zipcodes_4 = []
zip_pattern = r'\b\d{5}(?:-\d{4})?\b'

for address in addresses_4:
    match = re.search(zip_pattern, address)
    
    if match:
        zipcodes_4.append(match.group())
    else:
        zipcodes_4.append(0)
        
#print(zipcodes)
print(len(zipcodes_4))

#Now fetching the details of review_count using regex
reviews_4 = []

review_pattern = re.compile(r'\((\d+\+?) ratings\)')

#Iterate over output_list
for element in output_list_4:
    #Search for 'ratings' in the element
    match = review_pattern.search(element)
    if match:
        #If ratings found, extract the number within the brackets
        review = match.group(1)
        reviews_4.append(review)
    else:
        #If rating not found, return 'new'
        reviews_4.append('new')

#print(reviews)
print(len(reviews_4))

#Now extracting cuisine form this list
cuisine_list_4 = []

#Iterate over each element in output_list
for element in output_list_4:
    #Search for the substring after 'Circle i' using regex
    match = re.search(r'Circle i(\w+)(?=\W|$)', element)
    if match:
        cuicine_info = match.group(1)
        #split the cuicine info by non-alphanumeric characters and filter out empty strings
        cuisines = [c.strip() for c in re.split(r'\W+', cuicine_info) if c.strip()]
        cuisine_list_4.append(cuisines)
   
#print(cuisine_list)
print(len(cuisine_list_4))

#Now extracting the rating informaation using regex 
rating_pattern = r'(\d+(\.\d+)?)\s*\([0-9]+\s*ratings\)'

ratings_4 = []
#Iterate over output_list
for element in output_list_4:
    match = re.search(rating_pattern, element)
    if match:
        rating = match.group(1)
        ratings_4.append(rating)
    else:
        ratings_4.append('0')

#print(ratings)
print(len(ratings_4))

3
3
3
3
3
3


In [56]:
#Creating unique id
id_4 = [uuid.uuid4().hex[:10] for _ in range(3)]

In [57]:
# Function to create a SQL table
def create_sql_table(db_name, table_name, table_info):
    try:
        # Connect to server
        conn = mysql.connector.connect(host='127.0.0.1',
                                       port = '3306',
                                       user='root',
                                       password='Pvv@399051')
        cursor = conn.cursor()
        
        # Create database if it doesn't exist
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        
        # Create table if it doesn't exist
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {db_name}.{table_name} {table_info}")
        
    except mysql.connector.Error as err:
        print("Something went wrong:", err)
    finally:
        cursor.close()
        conn.close()

# Function to insert data into the table
def insert_data_dict(db_name, table_name, data_dict):
    try:
        #Connect to MySQL database
        conn = mysql.connector.connect(host='127.0.0.1',
                                       port = '3306',
                                       user='root',
                                       password='Pvv@399051', database=db_name)
        cursor = conn.cursor()
        
        #Find the max length among all values
        max_length = max(len(lst) if isinstance(lst,list) else 1 for lst in data_dict.values())
        
        #Fill missing values with None or empty string
        for key, value in data_dict.items():
            if not isinstance(value,list):
                data_dict[key] = [value] * max_length
            elif len(value) < max_length:
                data_dict[key].extend([None]*(max_legth - len(values)))
                
        #Create a parameterised sql query
        placeholders = ', '.join(['%s'] * len(data_dict))
        columns = ', '.join(data_dict.keys())
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        
        #Prepare the data values to be inserted
        values = [tuple(data_dict[key][i] for key in data_dict) for i in range(max_length)]
        
        #Execute the query with the values from the data dictionary
        cursor.executemany(sql, values)

        #Commit the transaction
        conn.commit()

    except mysql.connector.Error as err:
        print("Something went wrong:", err)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()


# My database and table name
SQL_DB = "bax422_final_price1"
SQL_TABLE_NAME = "uber_eats_price4"

# SQL table structure
SQL_TABLE_USER_INFO = """
(
    id VARCHAR(100) PRIMARY KEY,
    name VARCHAR(500),
    address1 VARCHAR(500),
    address2 VARCHAR(100),
    city VARCHAR(100),
    zip_code INT,
    review_count VARCHAR(100),
    categories VARCHAR(500),
    rating INT,
    price VARCHAR(10)
    
)
"""

# Create your database and table
create_sql_table(SQL_DB, SQL_TABLE_NAME, SQL_TABLE_USER_INFO)



#Converting dataframe uber_eats_1 to dictionary to  pass it through insert_into function
#uber_eats_1_dict = uber_eats_1.to_dict(orient = 'records')

#Insert data into the databse
for i in range(3):
    data_dict = {
        'id' : id_4[i],
        'name' : retaurant_names_4[i],
        'address1' : addresses_4[i],
        'address2' : '',
        'city' : ' San Francisco',
        'zip_code' : zipcodes_4[i],
        'review_count' : reviews_4[i],
        'categories' : cuisine_list_4[i],
        'rating': ratings_4[i],
        'price' :'$$$$'

    }
    insert_data_dict(SQL_DB, SQL_TABLE_NAME, data_dict)
